In [ ]:
import os
import numpy as np
import pandas as pd
import json
import pickle as cp
import matplotlib.pyplot as plt
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from tensorflow import keras
import tensorflow as tf
from keras import backend as K
import torch

In [ ]:
def getScore(model,X):
    input = tf.convert_to_tensor(X)
    with tf.GradientTape() as tape:
      tape.watch(input)
      output = model(input)
    gradients = np.absolute(tape.gradient(output, input))
    init = 0
    monthly_gradients = []
    for days in mnth_days:
        mgrad = gradients[init:init+days]
        monthly_gradients.append(mgrad.sum(axis=1).sum(axis=0)/days)
        init=init+days
    monthly_gradients = np.array(monthly_gradients)

    return monthly_gradients

In [ ]:
def addSpanLag(data, events=[], look_back=7, look_ahead = 7, season=0,span=0):
    ### Adding span (required for weekly, monthly average data)
    if span>1:
        nts = []
        ind = 0
        while ind < len(ts):
            tmpts = ts[ind:ind+span]
            print (tmpts)
            dt = tmpts[0]
            vals = [k for k in tmpts]
            pr = sum(vals)/len(vals)
            nts.append((dt,pr))
            ind+=span
        ts = nts


    ### Adding lag (based on look back and look ahead)

    if len(events) > 0:
        input = np.append(data,events,axis=1)
    else:
        input = data
    if season>0:
        season = np.array([k%season for k in range(data.shape[0])])
        season = season.reshape(len(season),1)
        input = np.append(input,season,axis=1)


    ind = 0
    X_data = []
    y_data = []

    while ind + look_back + look_ahead < len(data):

        X = input[ind:ind+look_back]
        Y = data[ind+look_back+look_ahead]
        X_data.append(X)
        y_data.append(Y)
        ind+=1
    X_data = np.array(X_data)
    y_data = np.array(y_data)

    return X_data, y_data

In [ ]:
crops = ['Onion','Potato','Rice','Wheat']
path='/home/sunchak/projects/news_analytics/REN/'
event_dict = {}
for crop in crops:

  model = keras.models.load_model(path+'onedayahead_logreturn/ren_'+crop)
  mnth_days = [31,28,31,30,31,30,31,31,30,31,30,31]*3
  fileformat = path+'food_price_data/%s.csv'
  data = pd.read_csv(fileformat%(crop))
  price = data['price']
  dates = data['date']
  #foodpriceDf = pd.Series(food_price,index=dates)
  start_date = '2012-12-24'
  end_date = '2015-12-31'
  price.index = dates
  range_price = price[start_date:end_date]

  food_price = np.log(range_price).diff().dropna()
  price_diff = np.array(food_price).reshape(food_price.shape[0],1)


  train = np.load(path+'embeddings/events_train.npy')
  test = np.load(path+'embeddings/events_test.npy')
  events = np.append(train,test, axis=0)[2548:2548+1102]

  X, y   = addSpanLag(price_diff,events,look_back=7,look_ahead=0,season=0)
  gradients = getScore(model,X)
  #gradients.shape
  evnt_grads = gradients[:,1:]
  top1 = []
  top10 = []
  for k in range(36):
    a = evnt_grads[k]
    ind = np.argpartition(a, -10)[-10:]
    top10.append(np.flip(ind))
    top1.append(ind[-1])
  event_dict[crop] = {}
  for evnt in top1:
    event_dict[crop][evnt] = evnt_grads[:,evnt]
cp.dump(event_dict,open(os.path.join(path,'event importance','event_scores.dict'),'wb'))